# Convert Supervisely json to txt labels
All the labeling work was done in Supervise.ly.  Their json format needs to be converted to KITTI txt in the format:

```
kitti_formatted
  ├── images
  │   ├── IMAGE_0011.jpg
  │   ├── ......
  │   └── IMAGE_2697.jpg
  └── labels
      ├── IMAGE_0011.txt
      ├── ......
      └── IMAGE_2697.txt
```

The format is described here:  https://github.com/NVIDIA/DIGITS/tree/master/digits/extensions/data/objectDetection

We will need to: 
* Enumerate our Object Classes (Turn text in number 1..n)
* Convert the json to KITTI format
  * I think
* Convert to tfrecords (This splits train/val)
* Retrain


In [1]:
import supervisely_lib as sly  # Supervisely Python SDK
import json                    # Add Python JSON module for pretty-printing.

# Load the project meta-data.
# This does NOT load the images or their labeling data in memory.
project = sly.Project('../../../../../../data/PowerCells', sly.OpenMode.READ)

With an opened project, we can get some basic information: its name, location on disk, member datasets, the total number of images. We can also iterate over the datasets, and go over the images in each dataset.

In [12]:
print(project.meta)

ProjectMeta:
Object Classes
+------------+-----------+----------------+
|    Name    |   Shape   |     Color      |
+------------+-----------+----------------+
| Power_Cell | Rectangle | [248, 231, 28] |
+------------+-----------+----------------+
Tags
+---------+------------+-----------------+
|   Name  | Value type | Possible values |
+---------+------------+-----------------+
| Invalid |    none    |       None      |
|  Valid  |    none    |       None      |
+---------+------------+-----------------+



In [14]:
# Print basic project metadata.
print("Project name: ", project.name)
print("Project directory: ", project.directory)
print("Total images: ", project.total_items)
print("Dataset names: ", project.datasets.keys())
print("\n")

# What datasets and images are there and where are they on disk?
for dataset in project:
    print("Dataset: ", dataset.name)
    
    # A dataset item is a pair of an image and its annotation.
    # The annotation contains all the labeling information about
    # the image - segmentation masks, objects bounding boxes etc.
    # We will look at annotations in detail shortly.
    for item_name in dataset:
        print(item_name)
        img_path = dataset.get_img_path(item_name)
        ann_path = dataset.get_ann_path(item_name)
        print("  image: ", img_path)
        print("  ann:   ", ann_path)
        ann = sly.Annotation.load_json_file(ann_path, project.meta)

        print('\tLoaded annotation has {} labels and {} image tags.'.format(len(ann.labels), len(ann.img_tags)))
        print('\tLabel class names: ' + (', '.join(label.obj_class.name for label in ann.labels)))
        print('\tImage tags: ' + (', '.join(tag.get_compact_str() for tag in ann.img_tags)))
        for label in ann.labels:
            print('\tLabel class: {}; type: {}; label area: {}'.format(
                    label.obj_class.name,
                    label.geometry.geometry_name(),
                    label.geometry.area))
            bbox = label.geometry.to_bbox()
            print('\tLabel bounding box: [{}, {}, {}, {}]'.format(
                    bbox.top, bbox.left, bbox.bottom, bbox.right))
        for tag in label.tags:
            print('Tag: ' + tag.get_compact_str())
    print()

Project name:  PowerCells
Project directory:  ../../../../../../data/PowerCells
Total images:  567
Dataset names:  ['Filming Day 1 Images', 'Filming Day 2 Video', 'Filming Day 1 Video']


Dataset:  Filming Day 1 Images
DSC02139.jpg
  image:  ../../../../../../data/PowerCells/Filming Day 1 Images/img/DSC02139.jpg
  ann:    ../../../../../../data/PowerCells/Filming Day 1 Images/ann/DSC02139.jpg.json
	Loaded annotation has 7 labels and 1 image tags.
	Label class names: Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_Cell
	Image tags: Valid
	Label class: Power_Cell; type: rectangle; label area: 306.0
	Label bounding box: [360, 188, 376, 205]
	Label class: Power_Cell; type: rectangle; label area: 288.0
	Label bounding box: [360, 249, 375, 266]
	Label class: Power_Cell; type: rectangle; label area: 289.0
	Label bounding box: [358, 311, 374, 327]
	Label class: Power_Cell; type: rectangle; label area: 754.0
	Label bounding box: [422, 76, 447, 104]
	Label class: Po

	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_03120.png
  image:  ../../../../../../data/PowerCells/Filming Day 2 Video/img/frame_03120.png
  ann:    ../../../../../../data/PowerCells/Filming Day 2 Video/ann/frame_03120.png.json
	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_02570.png
  image:  ../../../../../../data/PowerCells/Filming Day 2 Video/img/frame_02570.png
  ann:    ../../../../../../data/PowerCells/Filming Day 2 Video/ann/frame_02570.png.json
	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_02955.png
  image:  ../../../../../../data/PowerCells/Filming Day 2 Video/img/frame_02955.png
  ann:    ../../../../../../data/PowerCells/Filming Day 2 Video/ann/frame_02955.png.json
	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_03450.png
  image:  ../../../../../../data/PowerCells/Filming Day 2

	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_02695.png
  image:  ../../../../../../data/PowerCells/Filming Day 2 Video/img/frame_02695.png
  ann:    ../../../../../../data/PowerCells/Filming Day 2 Video/ann/frame_02695.png.json
	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_00055.png
  image:  ../../../../../../data/PowerCells/Filming Day 2 Video/img/frame_00055.png
  ann:    ../../../../../../data/PowerCells/Filming Day 2 Video/ann/frame_00055.png.json
	Loaded annotation has 4 labels and 1 image tags.
	Label class names: Power_Cell, Power_Cell, Power_Cell, Power_Cell
	Image tags: Valid
	Label class: Power_Cell; type: rectangle; label area: 729.0
	Label bounding box: [53, 341, 79, 367]
	Label class: Power_Cell; type: rectangle; label area: 676.0
	Label bounding box: [45, 370, 70, 395]
	Label class: Power_Cell; type: rectangle; label area: 702.0
	Label bounding box: [37, 398, 63, 423]
	La